In [ ]:
# Lesson1：Backtrader来啦
# link: https://mp.weixin.qq.com/s/7S4AnbUfQy2kCZhuFN1dZw


In [ ]:
import backtrader as bt
import pandas as pd
import datetime


daily_price = pd.read_csv("Data/daily_price.csv", parse_dates=['datetime'])  # 行情数据
trade_info = pd.read_csv("Data/trade_info.csv", parse_dates=['trade_date'])  # 调仓信息
print(daily_price.info())
print(trade_info.info())

In [ ]:
stock_code = ['600466.SH', '603228.SH']
# 过滤指定股票数据
daily_price = daily_price[daily_price['sec_code'].isin(stock_code)]
trade_info = trade_info[trade_info['sec_code'].isin(stock_code)]
print(daily_price.shape)
print(trade_info.shape)

In [17]:
# 按股票代码，依次循环传入数据,并保存指定股票代码到文件
stock_code = ['600466.SH', '603228.SH']
for code in stock_code:
    # 日期对齐
    data = pd.DataFrame(daily_price['datetime'].unique(), columns=['datetime'])  # 获取回测区间内所有交易日
    df = daily_price.query(f"sec_code=='{code}'")[
        ['datetime', 'open', 'high', 'low', 'close', 'volume', 'openinterest']]
    if df.empty:
        print(code + "不存在")
        continue
    data_ = pd.merge(data, df, how='left', on='datetime')
    data_ = data_.set_index("datetime")
    # print(data_.dtypes, data_.head())
    print(data_.info())
    # 缺失值处理：日期对齐时会使得有些交易日的数据为空，所以需要对缺失数据进行填充
    data_.loc[:, ['open', 'high', 'low', 'close']].dropna(inplace=True)
    data_.to_csv(
        path_or_buf='./Data/' + code + '-day.csv',
        sep=',',  # 分隔符
        header=True,  # 导出列标签
        date_format='%Y-%m-%d',  # 时期格式化字符串
        float_format='%.2f',  # 浮点数格式化字符串
    )

    print(f"{code} Done !")

print("All stock Done !")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 506 entries, 2019-01-02 to 2021-01-28
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   open          506 non-null    float64
 1   high          506 non-null    float64
 2   low           506 non-null    float64
 3   close         506 non-null    float64
 4   volume        506 non-null    int64  
 5   openinterest  506 non-null    int64  
dtypes: float64(4), int64(2)
memory usage: 27.7 KB
None
600466.SH Done !
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 506 entries, 2019-01-02 to 2021-01-28
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   open          506 non-null    float64
 1   high          506 non-null    float64
 2   low           506 non-null    float64
 3   close         506 non-null    float64
 4   volume        506 non-null    int64  
 5   openinterest  506 non-null    i

In [18]:
# 实例化 cerebro
cerebro = bt.Cerebro()

# 按股票代码，依次循环传入数据
stock_stock = ['600466.SH', '603228.SH']
daily_price_u = daily_price['sec_code'].unique()
for stock in stock_stock:
    # 日期对齐
    data = pd.DataFrame(daily_price['datetime'].unique(), columns=['datetime'])  # 获取回测区间内所有交易日
    df = daily_price.query(f"sec_code=='{stock}'")[
        ['datetime', 'open', 'high', 'low', 'close', 'volume', 'openinterest']]
    data_ = pd.merge(data, df, how='left', on='datetime')
    data_ = data_.set_index("datetime")
    # print(data_.dtypes)
    # 缺失值处理：日期对齐时会使得有些交易日的数据为空，所以需要对缺失数据进行填充
    data_.loc[:, ['volume', 'openinterest']] = data_.loc[:, ['volume', 'openinterest']].fillna(0)
    data_.loc[:, ['open', 'high', 'low', 'close']] = data_.loc[:, ['open', 'high', 'low', 'close']].fillna(method='pad')
    data_.loc[:, ['open', 'high', 'low', 'close']] = data_.loc[:, ['open', 'high', 'low', 'close']].fillna(0)
    # 导入数据
    datafeed = bt.feeds.PandasData(dataname=data_, fromdate=datetime.datetime(2019, 1, 2),
                                   todate=datetime.datetime(2021, 1, 28))
    cerebro.adddata(datafeed, name=stock)  # 通过 name 实现数据集与股票的一一对应
    print(f"{stock} Done !")

print("All stock Done !")


600466.SH Done !
603228.SH Done !
All stock Done !


In [19]:

# 回测策略
class TestStrategy(bt.Strategy):
    """选股策略"""
    params = (('maperiod', 15),
              ('printlog', False),)

    def __init__(self):
        self.buy_stock = trade_info  # 保留调仓列表
        # 读取调仓日期，即每月的最后一个交易日，回测时，会在这一天下单，然后在下一个交易日，以开盘价买入
        self.trade_dates = pd.to_datetime(self.buy_stock['trade_date'].unique()).tolist()
        self.order_list = []  # 记录以往订单，方便调仓日对未完成订单做处理
        self.buy_stocks_pre = []  # 记录上一期持仓

    def next(self):
        dt = self.datas[0].datetime.date(0)  # 获取当前的回测时间点
        # 如果是调仓日，则进行调仓操作
        if dt in self.trade_dates:
            print("--------------{} 为调仓日----------".format(dt))
            # 在调仓之前，取消之前所下的没成交也未到期的订单
            if len(self.order_list) > 0:
                for od in self.order_list:
                    self.cancel(od)  # 如果订单未完成，则撤销订单
                self.order_list = []  # 重置订单列表
            # 提取当前调仓日的持仓列表
            buy_stocks_data = self.buy_stock.query(f"trade_date=='{dt}'")
            long_list = buy_stocks_data['sec_code'].tolist()
            print('long_list', long_list)  # 打印持仓列表
            # 对现有持仓中，调仓后不再继续持有的股票进行卖出平仓
            sell_stock = [i for i in self.buy_stocks_pre if i not in long_list]
            print('sell_stock', sell_stock)  # 打印平仓列表
            if len(sell_stock) > 0:
                print("-----------对不再持有的股票进行平仓--------------")
                for _stock in sell_stock:
                    _data = self.getdatabyname(_stock)
                    if self.getposition(_data).size > 0:
                        od = self.close(data=_data)
                        self.order_list.append(od)  # 记录卖出订单
            # 买入此次调仓的股票：多退少补原则
            print("-----------买入此次调仓期的股票--------------")
            for _stock in long_list:
                w = buy_stocks_data.query(f"sec_code=='{_stock}'")['weight'].iloc[0]  # 提取持仓权重
                _data = self.getdatabyname(_stock)
                order = self.order_target_percent(data=_data, target=w * 0.95)  # 为减少可用资金不足的情况，留 5% 的现金做备用
                self.order_list.append(order)

            self.buy_stocks_pre = long_list  # 保存此次调仓的股票列表

        # 交易记录日志（可省略，默认不输出结果）

    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()},{txt}')

    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 已经处理的订单
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f，Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                    (order.ref,  # 订单编号
                     order.executed.price,  # 成交价
                     order.executed.value,  # 成交额
                     order.executed.comm,  # 佣金
                     order.executed.size,  # 成交量
                     order.data._name))  # 股票名称
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                         (order.ref,
                          order.executed.price,
                          order.executed.value,
                          order.executed.comm,
                          order.executed.size,
                          order.data._name))


# 初始资金 100,000,000
cerebro.broker.setcash(100000000.0)
# 佣金，双边各 0.0003
cerebro.broker.setcommission(commission=0.0003)
# 滑点：双边各 0.0001
cerebro.broker.set_slippage_perc(perc=0.0001)

cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='pnl')  # 返回收益率时序数据
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn')  # 年化收益率
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio')  # 夏普比率
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown')  # 回撤

# 将编写的策略添加给大脑，别忘了 ！
cerebro.addstrategy(TestStrategy, printlog=True)

# 启动回测
result = cerebro.run()
# 从返回的 result 中提取回测结果
strat = result[0]
# 返回日度收益率序列
daily_return = pd.Series(strat.analyzers.pnl.get_analysis())
# 打印评价指标
print("--------------- AnnualReturn -----------------")
print(strat.analyzers._AnnualReturn.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers._SharpeRatio.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers._DrawDown.get_analysis())



C:\Users\yalin\AppData\Local\Temp/ipykernel_15164/1869840904.py:17: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if dt in self.trade_dates:


--------------2019-02-28 为调仓日----------
long_list ['600466.SH', '603228.SH']
sell_stock []
-----------买入此次调仓期的股票--------------
2019-03-01,BUY EXECUTED, ref:9467，Price: 38.68, Cost: 1564509.19, Comm 469.35, Size: 40445.00, Stock: 600466.SH
2019-03-01,BUY EXECUTED, ref:9468，Price: 66.26, Cost: 438933.18, Comm 131.68, Size: 6624.00, Stock: 603228.SH
--------------2019-03-29 为调仓日----------
long_list ['600466.SH']
sell_stock ['603228.SH']
-----------对不再持有的股票进行平仓--------------
-----------买入此次调仓期的股票--------------
2019-04-01,SELL EXECUTED, ref:9469, Price: 68.36, Cost: 438933.18, Comm 135.84, Size: -6624.00, Stock: 603228.SH
2019-04-01,SELL EXECUTED, ref:9470, Price: 46.14, Cost: 245052.93, Comm 87.69, Size: -6335.00, Stock: 600466.SH
--------------2019-04-30 为调仓日----------
long_list ['600466.SH']
sell_stock []
-----------买入此次调仓期的股票--------------
2019-05-06,BUY EXECUTED, ref:9471，Price: 41.27, Cost: 220708.89, Comm 66.21, Size: 5348.00, Stock: 600466.SH
--------------2019-05-31 为调仓日----------
